In [1]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import timeit
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

2023-05-16 02:09:49.243915: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 02:09:49.374460: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-16 02:09:49.728936: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/themandalorian/anaconda3/envs/ml-on-mc/lib/
2023-05-16 02:09:49.728988: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: lib

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Load the CIFAR-10 dataset
data_path = 'cifar-10-batches-py/'
train_data = []
train_labels = []
for i in range(1, 6):
    batch = unpickle(data_path + 'data_batch_' + str(i))
    train_data.append(batch[b'data'])
    train_labels += batch[b'labels']
    
    
train_data = np.concatenate(train_data)
train_data = train_data.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data = unpickle(data_path + 'test_batch')[b'data']
test_data = test_data.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels = np.array(train_labels)
test_labels = np.array(unpickle(data_path + 'test_batch')[b'labels'])

# Print the number of features, classes, and training samples for CIFAR-10
print("CIFAR-10")
print("Number of features:", train_data.shape[1:])
print("Number of classes:", len(np.unique(train_labels)))
print("Number of training samples:", train_data.shape[0])

# Shuffle your dataset
train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
test_data, test_labels = shuffle(test_data, test_labels, random_state=42)

# Normalize pixel values
x_train = train_data.astype('float32') / 255.0
x_test = test_data.astype('float32') / 255.0

# Convert labels to categorical format
y_train = to_categorical(train_labels, num_classes=10)
y_test = to_categorical(test_labels, num_classes=10)

# create data generator
datagen = ImageDataGenerator(zoom_range=[0.9,1.1],
                             rotation_range=30,
                             brightness_range=[0.9,1.1],
                             width_shift_range=0.3, 
                             height_shift_range=0.3,
                             vertical_flip=True,
                             horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(x_train, y_train, batch_size=64)
# fit model
steps = int(x_train.shape[0] / 64)

CIFAR-10
Number of features: (32, 32, 3)
Number of classes: 10
Number of training samples: 50000


In [5]:
model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

model_10.summary()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI.h5', save_freq='epoch')

# compile model
opt = SGD(lr=0.001, momentum=0.9)
model_10.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 16, 16, 32)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_9 (Conv2D)           (None, 16, 16, 64)        36928     
                                                      

In [6]:
# Model Fit
model_10.fit(x_train,y_train,batch_size=64, epochs=400, validation_data=(x_test, y_test),callbacks=[checkpoint_callback])

Epoch 1/400
782/782 [==============================] - 5s 6ms/step - loss: 2.2784 - accuracy: 0.1420 - val_loss: 2.1875 - val_accuracy: 0.2019
Epoch 2/400
782/782 [==============================] - 4s 5ms/step - loss: 2.0951 - accuracy: 0.2091 - val_loss: 1.9812 - val_accuracy: 0.2808
Epoch 3/400
782/782 [==============================] - 4s 5ms/step - loss: 1.9535 - accuracy: 0.2550 - val_loss: 1.8115 - val_accuracy: 0.3392
Epoch 4/400
782/782 [==============================] - 4s 5ms/step - loss: 1.8145 - accuracy: 0.3055 - val_loss: 1.6783 - val_accuracy: 0.3879
Epoch 5/400
782/782 [==============================] - 4s 5ms/step - loss: 1.7319 - accuracy: 0.3442 - val_loss: 1.5773 - val_accuracy: 0.4230
Epoch 6/400
782/782 [==============================] - 4s 5ms/step - loss: 1.6721 - accuracy: 0.3681 - val_loss: 1.5248 - val_accuracy: 0.4430
Epoch 7/400
782/782 [==============================] - 4s 5ms/step - loss: 1.6134 - accuracy: 0.3972 - val_loss: 1.4887 - val_accuracy: 0.4596

Epoch 58/400
782/782 [==============================] - 4s 5ms/step - loss: 0.7508 - accuracy: 0.7382 - val_loss: 0.6906 - val_accuracy: 0.7582
Epoch 59/400
782/782 [==============================] - 4s 5ms/step - loss: 0.7419 - accuracy: 0.7410 - val_loss: 0.6445 - val_accuracy: 0.7711
Epoch 60/400
782/782 [==============================] - 4s 5ms/step - loss: 0.7348 - accuracy: 0.7451 - val_loss: 0.6373 - val_accuracy: 0.7789
Epoch 61/400
782/782 [==============================] - 4s 5ms/step - loss: 0.7308 - accuracy: 0.7473 - val_loss: 0.6453 - val_accuracy: 0.7728
Epoch 62/400
782/782 [==============================] - 4s 5ms/step - loss: 0.7272 - accuracy: 0.7478 - val_loss: 0.6464 - val_accuracy: 0.7751
Epoch 63/400
782/782 [==============================] - 4s 5ms/step - loss: 0.7170 - accuracy: 0.7521 - val_loss: 0.6387 - val_accuracy: 0.7792
Epoch 64/400
782/782 [==============================] - 4s 5ms/step - loss: 0.7115 - accuracy: 0.7529 - val_loss: 0.6316 - val_accuracy:

Epoch 115/400
782/782 [==============================] - 4s 5ms/step - loss: 0.5228 - accuracy: 0.8218 - val_loss: 0.5215 - val_accuracy: 0.8253
Epoch 116/400
782/782 [==============================] - 4s 6ms/step - loss: 0.5191 - accuracy: 0.8200 - val_loss: 0.5311 - val_accuracy: 0.8223
Epoch 117/400
782/782 [==============================] - 4s 5ms/step - loss: 0.5203 - accuracy: 0.8200 - val_loss: 0.5388 - val_accuracy: 0.8182
Epoch 118/400
782/782 [==============================] - 4s 5ms/step - loss: 0.5113 - accuracy: 0.8212 - val_loss: 0.5261 - val_accuracy: 0.8232
Epoch 119/400
782/782 [==============================] - 4s 5ms/step - loss: 0.5132 - accuracy: 0.8227 - val_loss: 0.5072 - val_accuracy: 0.8262
Epoch 120/400
782/782 [==============================] - 4s 5ms/step - loss: 0.5067 - accuracy: 0.8257 - val_loss: 0.5087 - val_accuracy: 0.8299
Epoch 121/400
782/782 [==============================] - 4s 5ms/step - loss: 0.5108 - accuracy: 0.8251 - val_loss: 0.5161 - val_ac

782/782 [==============================] - 4s 5ms/step - loss: 0.4100 - accuracy: 0.8587 - val_loss: 0.4982 - val_accuracy: 0.8404
Epoch 172/400
782/782 [==============================] - 4s 5ms/step - loss: 0.4067 - accuracy: 0.8607 - val_loss: 0.4789 - val_accuracy: 0.8450
Epoch 173/400
782/782 [==============================] - 4s 5ms/step - loss: 0.4052 - accuracy: 0.8595 - val_loss: 0.4954 - val_accuracy: 0.8402
Epoch 174/400
782/782 [==============================] - 4s 5ms/step - loss: 0.4063 - accuracy: 0.8596 - val_loss: 0.4873 - val_accuracy: 0.8420
Epoch 175/400
782/782 [==============================] - 4s 5ms/step - loss: 0.4075 - accuracy: 0.8607 - val_loss: 0.4873 - val_accuracy: 0.8428
Epoch 176/400
782/782 [==============================] - 4s 5ms/step - loss: 0.4045 - accuracy: 0.8614 - val_loss: 0.4865 - val_accuracy: 0.8432
Epoch 177/400
782/782 [==============================] - 4s 5ms/step - loss: 0.3991 - accuracy: 0.8614 - val_loss: 0.4726 - val_accuracy: 0.8445

782/782 [==============================] - 4s 5ms/step - loss: 0.3341 - accuracy: 0.8830 - val_loss: 0.4721 - val_accuracy: 0.8526
Epoch 228/400
782/782 [==============================] - 4s 5ms/step - loss: 0.3381 - accuracy: 0.8831 - val_loss: 0.4710 - val_accuracy: 0.8512
Epoch 229/400
782/782 [==============================] - 4s 5ms/step - loss: 0.3332 - accuracy: 0.8827 - val_loss: 0.4942 - val_accuracy: 0.8464
Epoch 230/400
782/782 [==============================] - 4s 5ms/step - loss: 0.3329 - accuracy: 0.8854 - val_loss: 0.4672 - val_accuracy: 0.8542
Epoch 231/400
782/782 [==============================] - 4s 5ms/step - loss: 0.3328 - accuracy: 0.8846 - val_loss: 0.4885 - val_accuracy: 0.8475
Epoch 232/400
782/782 [==============================] - 4s 6ms/step - loss: 0.3282 - accuracy: 0.8871 - val_loss: 0.4669 - val_accuracy: 0.8523
Epoch 233/400
782/782 [==============================] - 4s 5ms/step - loss: 0.3280 - accuracy: 0.8868 - val_loss: 0.4967 - val_accuracy: 0.8462

782/782 [==============================] - 4s 5ms/step - loss: 0.2840 - accuracy: 0.9016 - val_loss: 0.4983 - val_accuracy: 0.8547
Epoch 284/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2860 - accuracy: 0.9007 - val_loss: 0.4709 - val_accuracy: 0.8553
Epoch 285/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2809 - accuracy: 0.9032 - val_loss: 0.4883 - val_accuracy: 0.8582
Epoch 286/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2777 - accuracy: 0.9046 - val_loss: 0.4751 - val_accuracy: 0.8576
Epoch 287/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2810 - accuracy: 0.9025 - val_loss: 0.4777 - val_accuracy: 0.8584
Epoch 288/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2808 - accuracy: 0.9028 - val_loss: 0.4676 - val_accuracy: 0.8599
Epoch 289/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2808 - accuracy: 0.9031 - val_loss: 0.4805 - val_accuracy: 0.8574

782/782 [==============================] - 4s 5ms/step - loss: 0.2493 - accuracy: 0.9133 - val_loss: 0.4782 - val_accuracy: 0.8619
Epoch 340/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2502 - accuracy: 0.9136 - val_loss: 0.5124 - val_accuracy: 0.8550
Epoch 341/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2506 - accuracy: 0.9130 - val_loss: 0.5148 - val_accuracy: 0.8540
Epoch 342/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2473 - accuracy: 0.9137 - val_loss: 0.5029 - val_accuracy: 0.8578
Epoch 343/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2470 - accuracy: 0.9133 - val_loss: 0.4905 - val_accuracy: 0.8583
Epoch 344/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2458 - accuracy: 0.9142 - val_loss: 0.4893 - val_accuracy: 0.8613
Epoch 345/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2442 - accuracy: 0.9140 - val_loss: 0.4781 - val_accuracy: 0.8618

782/782 [==============================] - 4s 5ms/step - loss: 0.2175 - accuracy: 0.9234 - val_loss: 0.5118 - val_accuracy: 0.8606
Epoch 396/400
782/782 [==============================] - 4s 6ms/step - loss: 0.2227 - accuracy: 0.9230 - val_loss: 0.5120 - val_accuracy: 0.8604
Epoch 397/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2222 - accuracy: 0.9224 - val_loss: 0.5082 - val_accuracy: 0.8618
Epoch 398/400
782/782 [==============================] - 4s 6ms/step - loss: 0.2201 - accuracy: 0.9226 - val_loss: 0.5105 - val_accuracy: 0.8565
Epoch 399/400
782/782 [==============================] - 4s 6ms/step - loss: 0.2255 - accuracy: 0.9222 - val_loss: 0.4898 - val_accuracy: 0.8642
Epoch 400/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2257 - accuracy: 0.9222 - val_loss: 0.4928 - val_accuracy: 0.8610


In [7]:
from tensorflow.keras.models import load_model

# load the pre-saved model
model_10 = load_model('model_10_gpu_Base-CubeAI.h5')

# test the accuracy of the model on the test set
loss, accuracy = model_10.evaluate(x_test, y_test)

# print the accuracy of the model on the test set
print('Test accuracy:', accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 0.4928 - accuracy: 0.8610
Test accuracy: 0.8610000014305115


In [12]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model_10)

# Convert the model to the TensorFlow Lite format with quantization
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

tflite_model_name = 'model_10_gpu_Base-CubeAI'
        
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name + '.tflite')
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of quantized to int8 model is {}%".format(accuracy_score*100))

INFO:tensorflow:Assets written to: /tmp/tmp0abixuph/assets


INFO:tensorflow:Assets written to: /tmp/tmp0abixuph/assets
/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-16 02:50:40.054263: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-16 02:50:40.054290: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-16 02:50:40.054965: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp0abixuph
2023-05-16 02:50:40.058443: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-16 02:50:40.058466: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp0abixuph
2023-05-16 02:50:40.066049: I tensorflow/

== Input details ==
name: serving_default_conv2d_6_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.int8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.int8'>
Accuracy of quantized to int8 model is 86.17%


In [20]:
import tensorflow_model_optimization as tfmot

model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# Convert the model to a quantization aware model
quant_aware_model = tfmot.quantization.keras.quantize_model(model_10)

quant_aware_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

quant_aware_model.summary()

# Train and evaluate the quantization aware model
quant_aware_model.fit(x_train,y_train, batch_size=64,epochs=400,validation_data=(x_test, y_test))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_6 (QuantizeL  (None, 32, 32, 3)        3         
 ayer)                                                           
                                                                 
 quant_conv2d_36 (QuantizeWr  (None, 32, 32, 32)       963       
 apperV2)                                                        
                                                                 
 quant_conv2d_37 (QuantizeWr  (None, 32, 32, 32)       9315      
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_18 (Qua  (None, 16, 16, 32)       1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_dropout_24 (QuantizeW  (None, 16, 16, 32)      

782/782 [==============================] - 12s 15ms/step - loss: 1.0051 - accuracy: 0.6446 - val_loss: 0.8704 - val_accuracy: 0.6915
Epoch 32/400
782/782 [==============================] - 12s 15ms/step - loss: 0.9873 - accuracy: 0.6507 - val_loss: 0.8555 - val_accuracy: 0.6951
Epoch 33/400
782/782 [==============================] - 12s 15ms/step - loss: 0.9757 - accuracy: 0.6556 - val_loss: 0.9234 - val_accuracy: 0.6730
Epoch 34/400
782/782 [==============================] - 12s 15ms/step - loss: 0.9589 - accuracy: 0.6607 - val_loss: 0.8877 - val_accuracy: 0.6894
Epoch 35/400
782/782 [==============================] - 12s 15ms/step - loss: 0.9527 - accuracy: 0.6641 - val_loss: 0.8735 - val_accuracy: 0.6917
Epoch 36/400
782/782 [==============================] - 12s 16ms/step - loss: 0.9334 - accuracy: 0.6719 - val_loss: 0.8388 - val_accuracy: 0.7055
Epoch 37/400
782/782 [==============================] - 12s 15ms/step - loss: 0.9296 - accuracy: 0.6718 - val_loss: 0.8079 - val_accuracy

782/782 [==============================] - 12s 15ms/step - loss: 0.6168 - accuracy: 0.7884 - val_loss: 0.6121 - val_accuracy: 0.7936
Epoch 88/400
782/782 [==============================] - 12s 16ms/step - loss: 0.6060 - accuracy: 0.7902 - val_loss: 0.5933 - val_accuracy: 0.7999
Epoch 89/400
782/782 [==============================] - 12s 15ms/step - loss: 0.6012 - accuracy: 0.7918 - val_loss: 0.6033 - val_accuracy: 0.7965
Epoch 90/400
782/782 [==============================] - 12s 15ms/step - loss: 0.6051 - accuracy: 0.7930 - val_loss: 0.5679 - val_accuracy: 0.8097
Epoch 91/400
782/782 [==============================] - 12s 15ms/step - loss: 0.6014 - accuracy: 0.7936 - val_loss: 0.5844 - val_accuracy: 0.8078
Epoch 92/400
782/782 [==============================] - 12s 16ms/step - loss: 0.5969 - accuracy: 0.7953 - val_loss: 0.5596 - val_accuracy: 0.8132
Epoch 93/400
782/782 [==============================] - 12s 16ms/step - loss: 0.5932 - accuracy: 0.7979 - val_loss: 0.5793 - val_accuracy

Epoch 143/400
782/782 [==============================] - 12s 16ms/step - loss: 0.4577 - accuracy: 0.8439 - val_loss: 0.5149 - val_accuracy: 0.8295
Epoch 144/400
782/782 [==============================] - 12s 16ms/step - loss: 0.4603 - accuracy: 0.8406 - val_loss: 0.4978 - val_accuracy: 0.8376
Epoch 145/400
782/782 [==============================] - 12s 16ms/step - loss: 0.4614 - accuracy: 0.8421 - val_loss: 0.4880 - val_accuracy: 0.8423
Epoch 146/400
782/782 [==============================] - 12s 16ms/step - loss: 0.4508 - accuracy: 0.8462 - val_loss: 0.4892 - val_accuracy: 0.8375
Epoch 147/400
782/782 [==============================] - 12s 16ms/step - loss: 0.4538 - accuracy: 0.8428 - val_loss: 0.5082 - val_accuracy: 0.8376
Epoch 148/400
782/782 [==============================] - 12s 16ms/step - loss: 0.4539 - accuracy: 0.8432 - val_loss: 0.5314 - val_accuracy: 0.8309
Epoch 149/400
782/782 [==============================] - 12s 16ms/step - loss: 0.4489 - accuracy: 0.8449 - val_loss: 0

Epoch 199/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3723 - accuracy: 0.8717 - val_loss: 0.4962 - val_accuracy: 0.8453
Epoch 200/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3736 - accuracy: 0.8710 - val_loss: 0.4805 - val_accuracy: 0.8450
Epoch 201/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3692 - accuracy: 0.8731 - val_loss: 0.4871 - val_accuracy: 0.8486
Epoch 202/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3671 - accuracy: 0.8724 - val_loss: 0.5146 - val_accuracy: 0.8423
Epoch 203/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3685 - accuracy: 0.8716 - val_loss: 0.4819 - val_accuracy: 0.8498
Epoch 204/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3628 - accuracy: 0.8726 - val_loss: 0.5165 - val_accuracy: 0.8438
Epoch 205/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3600 - accuracy: 0.8772 - val_loss: 0

Epoch 255/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3065 - accuracy: 0.8932 - val_loss: 0.4966 - val_accuracy: 0.8482
Epoch 256/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3065 - accuracy: 0.8946 - val_loss: 0.4974 - val_accuracy: 0.8486
Epoch 257/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3070 - accuracy: 0.8916 - val_loss: 0.4828 - val_accuracy: 0.8529
Epoch 258/400
782/782 [==============================] - 12s 16ms/step - loss: 0.3054 - accuracy: 0.8945 - val_loss: 0.5114 - val_accuracy: 0.8464
Epoch 259/400
782/782 [==============================] - 12s 16ms/step - loss: 0.3045 - accuracy: 0.8943 - val_loss: 0.4934 - val_accuracy: 0.8543
Epoch 260/400
782/782 [==============================] - 12s 16ms/step - loss: 0.2994 - accuracy: 0.8944 - val_loss: 0.4833 - val_accuracy: 0.8549
Epoch 261/400
782/782 [==============================] - 12s 16ms/step - loss: 0.3002 - accuracy: 0.8964 - val_loss: 0

Epoch 311/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2624 - accuracy: 0.9087 - val_loss: 0.4923 - val_accuracy: 0.8565
Epoch 312/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2633 - accuracy: 0.9085 - val_loss: 0.4909 - val_accuracy: 0.8561
Epoch 313/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2643 - accuracy: 0.9073 - val_loss: 0.4938 - val_accuracy: 0.8564
Epoch 314/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2624 - accuracy: 0.9079 - val_loss: 0.4882 - val_accuracy: 0.8592
Epoch 315/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2664 - accuracy: 0.9064 - val_loss: 0.5133 - val_accuracy: 0.8551
Epoch 316/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2657 - accuracy: 0.9088 - val_loss: 0.4892 - val_accuracy: 0.8579
Epoch 317/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2606 - accuracy: 0.9082 - val_loss: 0

Epoch 367/400
782/782 [==============================] - 12s 16ms/step - loss: 0.2373 - accuracy: 0.9183 - val_loss: 0.5072 - val_accuracy: 0.8600
Epoch 368/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2343 - accuracy: 0.9183 - val_loss: 0.5145 - val_accuracy: 0.8561
Epoch 369/400
782/782 [==============================] - 12s 16ms/step - loss: 0.2381 - accuracy: 0.9183 - val_loss: 0.4895 - val_accuracy: 0.8607
Epoch 370/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2275 - accuracy: 0.9204 - val_loss: 0.5184 - val_accuracy: 0.8553
Epoch 371/400
782/782 [==============================] - 12s 16ms/step - loss: 0.2304 - accuracy: 0.9194 - val_loss: 0.5163 - val_accuracy: 0.8593
Epoch 372/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2300 - accuracy: 0.9205 - val_loss: 0.4915 - val_accuracy: 0.8605
Epoch 373/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2288 - accuracy: 0.9206 - val_loss: 0

In [21]:
# evaluate the model on the test set
quant_loss, quant_acc = quant_aware_model.evaluate(x_test, y_test, verbose=0)
print('Quantization aware training loss: ', quant_loss)
print('Quantization aware training accuracy: ', quant_acc)

# convert the QAT model to a fully quantized model using TFLite
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant_int8_qat = converter.convert()

open("model_10_gpu_Base-CubeAI_qat_int8.tflite", "wb").write(tflite_model_quant_int8_qat)

tflite_interpreter = tf.lite.Interpreter("model_10_gpu_Base-CubeAI_qat_int8.tflite")
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of quantized to int8 model is {}%".format(accuracy_score*100))

Quantization aware training loss:  0.5177656412124634
Quantization aware training accuracy:  0.8587999939918518


INFO:tensorflow:Assets written to: /tmp/tmp0n580i81/assets


INFO:tensorflow:Assets written to: /tmp/tmp0n580i81/assets
/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-16 04:22:37.054845: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-16 04:22:37.054877: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-16 04:22:37.055016: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp0n580i81
2023-05-16 04:22:37.060650: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-16 04:22:37.060665: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp0n580i81
2023-05-16 04:22:37.082583: I tensorflow/

== Input details ==
name: serving_default_conv2d_36_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.uint8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.uint8'>
Accuracy of quantized to int8 model is 85.86%
